## 批量导出教师成绩分析表【糖葫芦图+箱图】★

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os   

#%matplotlib inline
plt.rcdefaults()
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['figure.figsize']=[14,10] #改变显示图形的大小

In [2]:
file_name='data/xscjb20190215.csv'
fields=['XNM','XQM','JSGH','JSXM','KCH','KCMC','BFZCJ']
df_cjb_js_term=pd.read_csv(file_name,usecols=fields,dtype={'XNM':np.object,'XQM':np.object})

d:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_cjb_js_term.tail()
df_cjb_js_term.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813515 entries, 0 to 3813514
Data columns (total 7 columns):
XNM      object
XQM      object
BFZCJ    float64
KCH      object
KCMC     object
JSGH     object
JSXM     object
dtypes: float64(1), object(6)
memory usage: 203.7+ MB


### 获取指定学期的数据（XNM='2018-2019'  XQM='3'）

In [5]:
df_cjb_sinterm=df_cjb_js_term[(df_cjb_js_term.XNM=='2018') & (df_cjb_js_term.XQM=='3')]   #176923 rows × 7 columns
#df_cjb_sinterm=df_cjb_js_term
df_cjb_sinterm.head()

,XNM,XQM,BFZCJ,KCH,KCMC,JSGH,JSXM
3636592,2018,3,60.0,02300083,宏观经济学,NaN,无
3636593,2018,3,86.0,02300083,宏观经济学,NaN,无
3636594,2018,3,60.0,09100101,大学信息技术,NaN,无
3636595,2018,3,60.0,T1000417,大学体育3（男子篮球）,NaN,无
3636596,2018,3,60.0,30300003,入学教育,NaN,无


#### 载入教师基本信息表

In [6]:
file_name_js='data/jsjbxxb20190214.csv'
df_teacher=pd.read_csv(file_name_js,usecols=['JGH','JG_ID'])
df_teacher.head()
df_teacher.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3132 entries, 0 to 3131
Data columns (total 2 columns):
JGH      3132 non-null object
JG_ID    2787 non-null object
dtypes: object(2)
memory usage: 49.0+ KB


#### 在成绩库中加入教师部门代码（只需执行1次）

In [7]:
df_cjb_sinterm=df_cjb_sinterm.join(df_teacher.set_index('JGH'),on='JSGH')
df_cjb_sinterm.head()

,XNM,XQM,BFZCJ,KCH,KCMC,JSGH,JSXM,JG_ID
3636592,2018,3,60.0,02300083,宏观经济学,NaN,无,NaN
3636593,2018,3,86.0,02300083,宏观经济学,NaN,无,NaN
3636594,2018,3,60.0,09100101,大学信息技术,NaN,无,NaN
3636595,2018,3,60.0,T1000417,大学体育3（男子篮球）,NaN,无,NaN
3636596,2018,3,60.0,30300003,入学教育,NaN,无,NaN


In [8]:
#获取部门代码
set_jgid=df_cjb_sinterm.JG_ID.drop_duplicates(keep='first')
set_jgid.sort_values(inplace=True)
set_jgid.values

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '12',
       '15', '16', '17', '24', '28', '29', '30', '32', '33', '34', '36',
       '37', '38', '39', '52', '99', nan], dtype=object)

In [16]:
def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value


def set_axis_style(ax, labels):
    ax.get_xaxis().set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('Sample name')

In [17]:
'''
list_bm=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '12',
       '15', '16', '17', '24', '28', '29', '30', '32', '33', '34', '36',
       '37', '38', '39', '52', '99']
'''
#部门代码 
list_bm=['01']

list_err=[]
#list_err1=['201000023','200500047','201700030','200800025','201200028']   #2018-2019-1  出错的教师工号
#list_err=['p00000074','p00000229','p00000016','910000404','p00000196','p00000040','p00000185',
#          'p00000186','p00000187','200900046','201700006']   #全部成绩导出出错
colors = ['yellow','pink', 'lightblue', 'lightgreen','lightyellow','goldenrod','darkolivegreen',
         'dodgerblue','darkcyan','lightgray','tomato','indianred','khaki','lime']

red_square = dict(markerfacecolor='r', marker='s')
#for jg_id in set_jgid:
for jg_id in list_bm:
    #建立部门代码目录
    path_name='img/thlt_xt/'+str(jg_id)
    isExists=os.path.exists(path_name)
    if not isExists:
        os.mkdir(path_name)

    ser_js_jgid=df_teacher[df_teacher.JG_ID==jg_id].JGH
    
    #糖葫芦图所用数据
    #df_cjb_group=df_cjb_sinterm.groupby(['JSGH','KCMC','BFZCJ']).count()
    #df_cjb_group.sort_values('KCMC')
    #df_cjb_group.reset_index(inplace=True)

    for index in ser_js_jgid.index:
        jsgh=ser_js_jgid[index]
        print(jg_id,jsgh)
        
        if jsgh in list_err:
            pass
        else :
            #df_cjb_single=df_cjb_group[df_cjb_group.JSGH==jsgh] 
            #len1=len(df_cjb_single.groupby('KCMC').count())
            #df_cjb_single.loc[0]=[jsgh,'A',0,1,1,1,1,1]     #人为插入一条无关记录，是为了箱图与糖葫芦图共享X轴，上下显示一致。
            #df_cjb_single=df_cjb_single.sort_values('KCMC')
                
#            if (len1 !=0):   #没有课程的教师不绘制图像
                fig=plt.figure(figsize=(6,10))    #设置图像大小
                (ax2,ax1)=fig.subplots(2,1)
                fig.suptitle("单个教师全部课程成绩分析")
                
                #----------绘制箱图----------------------
                df_cjb_xt=df_cjb_sinterm[df_cjb_sinterm.JSGH==jsgh]
                list_labels=df_cjb_xt.groupby('KCMC').count().sort_index(0).index.tolist()   #获取课程名称
                #print(list_labels)
                #生成箱图数据
                list_cjb_show=[]
                for kcmc in list_labels:
                    array_cjb=df_cjb_xt[df_cjb_xt.KCMC==kcmc].BFZCJ.to_numpy()
                    array_cjb.shape=(-1,1)    
                    list_cjb_show.append(array_cjb)

                ax2.grid()
                #if len1 ==0:
                #    pass
                #elif len1==1:
                #    show_data=list_cjb_show[0]
                #else :
                show_data=list_cjb_show

                dic_cjb_single=ax2.violinplot(show_data,widths=0.9,showmeans=True,showextrema=True,showmedians=True) 

                for pc in dic_cjb_single['bodies']:
                    pc.set_facecolor('#D43F3A')
                    pc.set_edgecolor('black')
                    pc.set_alpha(1)
 
                quartile1, medians, quartile3 = np.percentile(show_data, [25, 50, 75], axis=1)
                whiskers = np.array([
                    adjacent_values(sorted_array, q1, q3)
                    for sorted_array, q1, q3 in zip(show_data, quartile1, quartile3)])
                whiskersMin, whiskersMax = whiskers[:, 0], whiskers[:, 1]

                inds = np.arange(1, len(medians) + 1)
                ax2.scatter(inds, medians, marker='o', color='white', s=30, zorder=3)
                ax2.vlines(inds, quartile1, quartile3, color='k', linestyle='-', lw=5)
                ax2.vlines(inds, whiskersMin, whiskersMax, color='k', linestyle='-', lw=1)

                #ax2.plot([-1,len1+1],[60,60],'r-.')
                plt.yticks(range(0,101,10))
                
                pic_file_name='img/thlt_xt/'+jg_id+'/'+jsgh+'任课成绩分析.png'
                plt.savefig(pic_file_name)
                plt.close()     #否则打开的图形太多
                #plt.show()

print('ok')

01 200800090


AxisError: axis 1 is out of bounds for array of dimension 1

##### --end---